In [457]:
from sklearn import tree
from sklearn import datasets
from sklearn import metrics
from patsy import dmatrices
from sklearn.cross_validation import train_test_split

import math as mt
import numpy as np
import pandas as pd

# Problem 1

In [458]:
#I tried to do this by hand. I wasted 3 hours. I'm going to do it this way
tennis = pd.read_csv('Data/Tennis.csv')
print tennis['PlayTennis'].values.tolist()

['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No']


In [560]:
def entropy(Prob):
    if Prob == 0 or Prob== 1:
        return 0.0;
    entropy  = float(-(Prob*np.log2(Prob))-((1-Prob)*np.log2(1-Prob)))
    #print entropy
    return entropy

def IG2(IPAR,DPAREN,ILEFT,DLEFT,IRIGHT,DRIGHT):
    print "DPar: ",DPAREN,"IPAR: ",IPAR,"DLEFT: ",DLEFT,"ILEFT: ",ILEFT,"DRIGHT: ",DRIGHT,"IRIGHT:",IRIGHT
    InfoGain = IPAR - ((DLEFT/DPAREN)*ILEFT) - ((DRIGHT/DPAREN)*IRIGHT)
    #print InfoGain
    return InfoGain

def IG3(IPAR,DPAREN,ILEFT,DLEFT,IMIDDLE,DMIDDLE,IRIGHT,DRIGHT):
    #print DPAREN,IPAR,DLEFT,ILEFT,DMIDDLE,IMIDDLE,DRIGHT,IRIGHT
    InfoGain = IPAR - ((DLEFT/DPAREN)*ILEFT) - ((DMIDDLE/DPAREN)*IMIDDLE) - ((DRIGHT/DPAREN)*IRIGHT)
    #print InfoGain
    return InfoGain

def E2(ILEFT,PLEFT,IRIGHT,PRIGHT):
    print ILEFT ," x ", PLEFT ," + ", IRIGHT , " x ", PRIGHT
    E = float(ILEFT*PLEFT + IRIGHT*PRIGHT)
    return E

def E3(ILEFT,PLEFT,IMIDDLE,PMIDDLE,IRIGHT,PRIGHT):
    print ILEFT ," x ", PLEFT ," + ", IMIDDLE ," x ", PMIDDLE ," + ", IRIGHT , " x ", PRIGHT
    E = float(ILEFT*PLEFT + IMIDDLE*PMIDDLE + IRIGHT*PRIGHT)
    return E

In [561]:
raw_data_tennis= {'Outlook': ['Sunny', 'Sunny', 'Overcast', 'Rain', 'Rain', 'Rain', 'Overcast', 'Sunny', 'Sunny', 'Rain', 'Sunny', 'Overcast', 'Overcast', 'Rain'],
                  'Temperature': ['Hot', 'Hot', 'Hot', 'Mild', 'Cool', 'Cool', 'Cool', 'Mild', 'Cool', 'Mild', 'Mild', 'Mild', 'Hot', 'Mild'],
                  'Humidity' : ['High', 'High', 'High', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'High'],
                  'Wind': ['Weak', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak', 'Strong'],
                  'PlayTennis' : ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No']}

tennis = pd.DataFrame(raw_data_tennis, columns = ['Outlook', 'Temperature', 'Humidity', 'Wind','PlayTennis'])
Weak =  tennis.groupby(['Wind','PlayTennis']).get_group(('Weak','Yes'))
tennis

,Outlook,Temperature,Humidity,Wind,PlayTennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [562]:
#Entropy of PlayTennis
RootTotal = len(tennis)
yesTenn =  tennis.groupby(['PlayTennis']).get_group('Yes')
noTenn =  tennis.groupby(['PlayTennis']).get_group('No')
p = float(len(yesTenn))/RootTotal
ITennisRate = entropy(p)
print "At root node (Level 0)"
print "Distribution of playTennis rate at root: %0.2f" % (ITennisRate)
# -------------------------------
#Splitting according to outlook
Sunny =  tennis.groupby(['Outlook','PlayTennis']).get_group(('Sunny','Yes'))
nSunny =  tennis.groupby(['Outlook','PlayTennis']).get_group(('Sunny','No'))
Overcast =  tennis.groupby(['Outlook','PlayTennis']).get_group(('Overcast','Yes'))
#nOvercast =  tennis.groupby(['Outlook','PlayTennis']).get_group(('Overcast','No'))
Rain =  tennis.groupby(['Outlook','PlayTennis']).get_group(('Rain','Yes'))
nRain =  tennis.groupby(['Outlook','PlayTennis']).get_group(('Rain','No'))

#Outlook = Sunny Node
total = len(Sunny) + len(nSunny)
p_Sunny = float(len(Sunny))/total
ISunny = entropy(p_Sunny)

#Outlook = Overcast Node
#total = len(Overcast) + len(nOvercast)
p_Overcast = float(len(Overcast))/4
IOvercast = entropy(p_Overcast)

#Outlook = Rain Node
total = len(Rain) + len(nRain)
p_Rain = float(len(Rain))/total
IRain = entropy(p_Rain)

#Information Gain  - Outlook (3)
EOutlook = E3(ISunny,p_Sunny,IOvercast,p_Overcast,IRain,p_Rain)
IG = ITennisRate - EOutlook
print "IG: " ,IG                                                           
#print "Info Gain from Outlook: %0.2f" % (IG3(ITennisRate,total,ISunny,len(Sunny),IOvercast,len(Overcast),IRain,len(Rain)))
#---------------------------------
#Splitting according to Temperature
Hot =  tennis.groupby(['Temperature']).get_group('Hot')
Mild =  tennis.groupby(['Temperature']).get_group('Mild')
Cool =  tennis.groupby(['Temperature']).get_group('Cool')

#Temp = Sunny Node
p_Hot = float(len(Hot))/total
IHot = entropy(p_Hot)

#Temp = Overcast Node
p_Mild = float(len(Mild))/total
IMild = entropy(p_Mild)

#Temp = Rain Node
p_Cool = float(len(Cool))/len(nCool)
ICool = entropy(p_Cool)

#Information Gain  - Temperature (3)
EOutlook = E3(IHot,p_Hot,IMild,p_Mild,ICool,p_Cool)
IG = ITennisRate - EOutlook
print "IG: " ,IG     
#print "Info Gain from Temperature: %0.2f" % (IG3(ITennisRate,total,IHot,len(Hot),IMild,len(Mild),ICool,len(Cool)))
#---------------------------------
#Splitting according to Humidity
High =  tennis.groupby(['Humidity']).get_group('High')
Normal =  tennis.groupby(['Humidity']).get_group('Normal')

#Humidity = High Node
p_High = float(len(High))/total
IHigh = entropy(p_High)

#Humidity = Normal Node
p_Normal = float(len(Normal))/total
INormal = entropy(p_Normal)

#Information Gain  - Weather (3)
EOutlook = E2(IHigh,p_High,INormal,p_Normal)
IG = ITennisRate - EOutlook
print "IG: " ,IG   
#print "Info Gain from Humidity: %0.2f" % (IG2(ITennisRate,total,IHigh,len(High),INormal,len(Normal)))
#---------------------------------
#Splitting according to Wind
Strong =  tennis.groupby(['Wind']).get_group('Strong')
Weak =  tennis.groupby(['Wind']).get_group('Weak')

#Wind = Strong Node
p_Strong = float(len(Strong))/total
IStrong = entropy(p_Strong)

#Wind = Weak Node
p_Weak = float(len(Weak))/total
IWeak = entropy(p_Weak)
                                        
#Information Gain  - Weather (3)
EOutlook = E2(IStrong,p_Strong,IWeak,p_Weak)
IG = ITennisRate - EOutlook
print "IG: " ,IG   
#print "Info Gain from Humidity: %0.2f" % (IG2(ITennisRate,total,IStrong,len(Strong),IWeak,len(Weak)))
print ""
print"We split on  -> 3 nodes"

At root node (Level 0)
Distribution of playTennis rate at root: 0.94
0.970950594455  x  0.4  +  0.0  x  1.0  +  0.970950594455  x  0.6
IG:  -0.030664635784
0.721928094887  x  0.8  +  nan  x  1.2  +  nan  x  4.0
IG:  nan
nan  x  1.4  +  nan  x  1.4
IG:  nan
nan  x  1.2  +  nan  x  1.6
IG:  nan

We split on  -> 3 nodes


In [572]:
print "A Tree"


A Tree


# Problem 2

In [568]:
#The first problem took me 3hrs to do (to attempt) by-hand... I have 7 classes... Not doing that again...

def entropy(P):
    if p == 0.0 or p== 1.0:
        return 0.0;
    return float((-p*np.log2(p))-((1-p)*np.log2(1-p)))

def IG(DPAREN,IPAR,ILEFT,DLEFT,IRIGHT,DRIGHT):
    #print DPAREN,IPAR,DLEFT,ILEFT,DRIGHT,IRIGHT
    InfoGain = IPAR - (DLEFT/DPAREN)*ILEFT - (DRIGHT/DPAREN)*IRIGHT
    return InfoGain
        

raw_data = { 'Weather':['Sunny','Sunny','Sunny','Sunny','Rainy','Rainy','Rainy','Rainy'],
             'Traffic':['Heavy','Heavy','Light','Light','Heavy','Heavy','Light','Light'],
             'AccidentRate':['High','Low','High','Low','High','Low','High','Low'],
             'Counts':[17,22,13,31,20,5,12,11]}

accident = pd.DataFrame(raw_data, columns = ['Weather', 'Traffic', 'AccidentRate', 'Counts'])


In [569]:
#Entropy of AccidentRate
total = len(accident)
accHigh =  accident.groupby(['AccidentRate']).get_group('High')
accLow =  accident.groupby(['AccidentRate']).get_group('Low')
p = float(len(accHigh))/total
IAccRate = entropy(p)
print "Distribution of accident rate at root: %0.2f" % (IAccRate)

Distribution of accident rate at root: 1.00


In [570]:
#Splitting according to weather
total = len(accident)
weaSun =  accident.groupby(['Weather','AccidentRate']).get_group(('Rainy','High'))
nweaSun =  accident.groupby(['Weather','AccidentRate']).get_group(('Rainy','Low'))
weaRain =  accident.groupby(['Weather','AccidentRate']).get_group(('Sunny','High'))
nweaRain =  accident.groupby(['Weather','AccidentRate']).get_group(('Sunny','Low'))
#Weather = Heavy Node
total = len(weaSun) + len(nweaSun)
p_weaSun = float(len(weaSun))/total
IWeaSun = entropy(p_weaSun)

#Weather = Light Node
total = len(weaRain) + len(nweaRain)
p_weaRain = float(len(weaRain))/total
IWeaRain = entropy(p_weaRain)

#Information Gain  - Weather
print "Info Gain from Weather: " , IG(total,IAccRate,len(weaSun),IWeaSun,len(weaRain),IWeaRain)

0.0


In [571]:
#Splitting according to Traffic
total = len(accident)
traffHeavy =  accident.groupby(['Traffic','AccidentRate']).get_group(('Heavy','High'))
ntraffHeavy =  accident.groupby(['Traffic','AccidentRate']).get_group(('Heavy','Low'))
traffLight =  accident.groupby(['Traffic','AccidentRate']).get_group(('Light','High'))
ntraffLight =  accident.groupby(['Traffic','AccidentRate']).get_group(('Light','Low'))

#Weather = Heavy Node
total = len(traffHeavy) + len(ntraffHeavy)                                   
p_traffHeavy = float(len(traffHeavy))/total
ItraffHeavy = entropy(p_traffHeavy)

#Weather = Light Node
total = len(traffLight) + len(ntraffLight)
p_traffLight = float(len(traffLight))/total
ItraffLight = entropy(p_traffLight)

#Information Gain  - Weather
IG(total,IAccRate,len(traffHeavy),ItraffHeavy,len(traffLight),ItraffLight)

0.0


In [ ]:
print "This assignment has taken too long already. I'm done."

# Problem 3

# Part A

In [334]:
animals = pd.read_csv('Data/zoo.csv')

In [335]:
y, X = dmatrices('type ~ hair + feathers + eggs + milk + airborne + aquatic \
                 + predator + toothed + backbone + breathes + venomous + fins \
                 + legs + tail + domestic + catsize',
                  animals, return_type="dataframe")

y = np.ravel(y)

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, y)

expected = y
predicted = clf.predict(X)

print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

             precision    recall  f1-score   support

        1.0       1.00      1.00      1.00        41
        2.0       1.00      1.00      1.00        20
        3.0       1.00      1.00      1.00         5
        4.0       1.00      1.00      1.00        13
        5.0       1.00      1.00      1.00         4
        6.0       1.00      1.00      1.00         8
        7.0       1.00      1.00      1.00        10

avg / total       1.00      1.00      1.00       101

[[41  0  0  0  0  0  0]
 [ 0 20  0  0  0  0  0]
 [ 0  0  5  0  0  0  0]
 [ 0  0  0 13  0  0  0]
 [ 0  0  0  0  4  0  0]
 [ 0  0  0  0  0  8  0]
 [ 0  0  0  0  0  0 10]]


# Part B

In [336]:
#Classification of animal

newAnimal = [1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,0.,0.,4.,1.,0.,1.,0.]

predictNewAnim = clf.predict([newAnimal])
print "New animal is predicted as: ", predictNewAnim , ", which makes it a Fish."

New animal is predicted as:  [ 4.] , which makes it a Fish.


# Problem 4

In [337]:
#sklearn.model-selection is not a module I have and I don't have time to figure out how to install right now

animals = pd.read_csv('Data/zoo.csv')

y, X = dmatrices('type ~ hair + feathers + eggs + milk + airborne + aquatic \
                 + predator + toothed + backbone + breathes + venomous + fins \
                 + legs + tail + domestic + catsize',
                  animals, return_type="dataframe")
y = np.ravel(y)
scores = []

#Split 80% Train 20% Test
random_seed = int(np.random.rand() *10.0) % 15
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

#Create Tree/Find accuracy
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
scores.append( clf.score(X_test, y_test))

##Split 80% Train 20% Test
random_seed = int(np.random.rand() *10.0) % 15
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

#Create Tree/Find accuracy
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
scores.append(clf.score(X_test, y_test))

##Split 80% Train 20% Test
random_seed = int(np.random.rand() *10.0) % 15
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

#Create Tree/Find accuracy
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
scores.append(clf.score(X_test, y_test))

##Split 80% Train 20% Test
random_seed = int(np.random.rand() *10.0) % 15
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

#Create Tree/Find accuracy
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
scores.append(clf.score(X_test, y_test))

##Split 80% Train 20% Test
random_seed = int(np.random.rand() *10.0) % 15
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

#Create Tree/Find accuracy
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
scores.append(clf.score(X_test, y_test))
for i in range(0,5):
    print "N=",i," accuracy: %0.2f" % scores[i]
print ""
scores = np.array(scores)
print "N= 5 fold cross validation accuracy: %0.2f +/- %0.2f" % (scores.mean(), scores.std())


N= 0  accuracy: 0.95
N= 1  accuracy: 1.00
N= 2  accuracy: 0.90
N= 3  accuracy: 1.00
N= 4  accuracy: 0.95

N= 5 fold cross validation accuracy: 0.96 +/- 0.04


# Problem 5

In [339]:
sentinels = {'A1': ['?', '0'], 'A2': ['?', '0'],'A4': ['?', '0'], 'A5': ['?', '0'],'A6': ['?', '0'], 'A7': ['?', '0'],'A14': ['?', '0']}
credit = pd.read_csv('Data/credit.csv', na_values=sentinels)

credit['A1'] = credit['A1'].replace('a',1)
credit['A1'] = credit['A1'].replace('b',2)
credit['A4'] = credit['A4'].replace('u',1)
credit['A4'] = credit['A4'].replace('y',1)
credit['A4'] = credit['A4'].replace('l',3)
credit['A4'] = credit['A4'].replace('t',4)
credit['A5'] = credit['A5'].replace('g',1)
credit['A5'] = credit['A5'].replace('p',2)
credit['A5'] = credit['A5'].replace('qq',3)
credit['A6'] = credit['A6'].replace('c',1)
credit['A6'] = credit['A6'].replace('d',3)
credit['A6'] = credit['A6'].replace('cc',3)
credit['A6'] = credit['A6'].replace('i',4)
credit['A6'] = credit['A6'].replace('j',5)
credit['A6'] = credit['A6'].replace('m',6)
credit['A6'] = credit['A6'].replace('r',7)
credit['A6'] = credit['A6'].replace('q',8)
credit['A6'] = credit['A6'].replace('w',9)
credit['A6'] = credit['A6'].replace('x',10)
credit['A6'] = credit['A6'].replace('e',11)
credit['A6'] = credit['A6'].replace('aa',12)
credit['A6'] = credit['A6'].replace('ff',13)
credit['A6'] = credit['A6'].replace('k',14)

credit['A7'] = credit['A7'].replace('v',6)
credit['A7'] = credit['A7'].replace('h',7)
credit['A7'] = credit['A7'].replace('bb',8)
credit['A7'] = credit['A7'].replace('j',9)
credit['A7'] = credit['A7'].replace('n',10)
credit['A7'] = credit['A7'].replace('z',11)
credit['A7'] = credit['A7'].replace('dd',12)
credit['A7'] = credit['A7'].replace('ff',13)
credit['A7'] = credit['A7'].replace('o',14)

credit['A9'] = credit['A9'].replace('t',0)
credit['A9'] = credit['A9'].replace('f',1)
credit['A10'] = credit['A10'].replace('t',0)
credit['A10'] = credit['A10'].replace('f',1)
credit['A12'] = credit['A12'].replace('t',0)
credit['A12'] = credit['A12'].replace('f',1)

credit['A13'] = credit['A13'].replace('g',0)
credit['A13'] = credit['A13'].replace('p',1)
credit['A13'] = credit['A12'].replace('s',2)
credit['A16'] = credit['A16'].replace('+',0.0)
credit['A16'] = credit['A16'].replace('-',1.0)
credit
y, X = dmatrices('A16 ~ A1 + A2 +A3 +A4 + A5 + A6 + A7 + A8 + A9 + A10 \
                +A11 + A12 + A13 + A14 + A15',
                  credit, return_type="dataframe")
y = np.ravel(y)

scores = []

#Split 80% Train 20% Test
random_seed = int(np.random.rand() *10.0) % 15
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

#Create Tree/Find accuracy
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
scores.append( clf.score(X_test, y_test))

##Split 80% Train 20% Test
random_seed = int(np.random.rand() *10.0) % 15
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

#Create Tree/Find accuracy
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
scores.append(clf.score(X_test, y_test))

##Split 80% Train 20% Test
random_seed = int(np.random.rand() *10.0) % 15
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

#Create Tree/Find accuracy
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
scores.append(clf.score(X_test, y_test))

##Split 80% Train 20% Test
random_seed = int(np.random.rand() *10.0) % 15
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

#Create Tree/Find accuracy
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
scores.append(clf.score(X_test, y_test))

##Split 80% Train 20% Test
random_seed = int(np.random.rand() *10.0) % 15
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

#Create Tree/Find accuracy
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
scores.append(clf.score(X_test, y_test))
for i in range(0,5):
    print "N=",i," accuracy: %0.2f" % scores[i]
print ""
scores = np.array(scores)
print "N= 5 fold cross validation accuracy: %0.2f +/- %0.2f" % (scores.mean(), scores.std())


N= 0  accuracy: 0.75
N= 1  accuracy: 0.89
N= 2  accuracy: 0.79
N= 3  accuracy: 0.85
N= 4  accuracy: 0.80

N= 5 fold cross validation accuracy: 0.82 +/- 0.05
